In [1]:
# Initialize Otter
import otter
grader = otter.Notebook("hw05.ipynb")

# Homework 5: SQL

## Due Date: Oct 2st, 11:59 PM

**Collaborators**: *list  collaborators here*

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sqlalchemy
from pathlib import Path

plt.style.use('fivethirtyeight') # Use plt.style.available to see more styles
sns.set()
sns.set_context("talk")
%matplotlib inline

The assignment consists of **8 problems** with 12 parts total. All questions are designed to look innocent, most with descriptions of 7 words or less. However, these questions get difficult, quickly, and reflect real-world queries you would find on the job:
- **1 warmup** (Q1): Just to get you up and running.
- **3 problems** (Q2, Q3, Q4): Relevant lectures have highly-similar practice problems.
- **3 guided, difficult problems** (Q5, Q6, Q7): Conceptual insight needed. Relevant lectures have somewhat similar practice problems.
- **1 brutally difficult but optional problem** (Q8)


<details>
    <summary>▶ [Click to Expand] <b>Logistics:</b> Collaboration Policy, Grading, Scoring Breakdown</summary>

## Collaboration Policy

Data science is a collaborative activity. While you may talk with others about
the homework, we ask that you **write your solutions individually**. If you do
discuss the assignments with others please **include their names** below.
    
## Grading

Grading is broken down into autograded answers and free response. 

For autograded answers, the results of your code are compared to provided and/or hidden tests. **For autograded probability questions, the provided tests will only check that your answer is within a reasonable range.**

### Please make sure to save regularly; do not rely on autosave to save your work for you!

**Caution:** Be careful with large SQL queries.  You may need to reboot your Jupyter Hub instance if it stops responding.

## Scoring Breakdown

|Question|Points|
|---|---|
|Q1 | 1 |
|Q2 | 2 |
|Q3 | 3 |
|Q4 | 3 |
|Q5 | 2 |
|Q6a | 2 |  
|Q6b | 3 |
|Q7a | 2 |
|Q7b | 3 |
|Total | 21 |
</details>

<details>
    <summary>▶ [Click to Expand] <b>Dataset</b>: In this assignment, we will work with a miniature version of the <a href="https://www.imdb.com/interfaces/">IMDB Dataset</a>. There are 4 tables: <code>Title</code>, <code>Rating</code>, <code>Name</code>, <code>Role</code>. Click here to see description of 4 tables.</b></summary>

    
**`Name`** – Contains the following information for names (originally called `names.basic`)
- nconst (string) - alphanumeric unique identifier of the name/person
- primaryName (string)– name by which the person is most often credited
- birthYear – in YYYY format
- deathYear – in YYYY format if applicable, else '\\N'
- primaryProfession (array of strings)– the top-3 professions of the person
- knownForTitles (array of tconsts) – titles the person is known for
    
    
**`Rating`** – Contains the IMDb rating and votes information for titles
- tconst (string) - alphanumeric unique identifier of the title
- averageRating – weighted average of all the individual user ratings
- numVotes - number of votes the title has received
    
    
**`Role`** – Contains the principal cast/crew for titles (originally called `principals`)
- tconst (string) - alphanumeric unique identifier of the title
- ordering (integer) – a number to uniquely identify rows for a given titleId
- nconst (string) - alphanumeric unique identifier of the name/person
- category (string) - the category of job that person was in
- job (string) - the specific job title if applicable, else '\\N'
- characters (string) - the name of the character played if applicable, else '\\N'
    
    
**`Title`** - Contains the following information for titles (originally called `title.basics`):
- tconst (string) - alphanumeric unique identifier of the title
- titleType (string) – the type/format of the title (e.g. movie, short, tvseries, tvepisode, video, etc)
- primaryTitle (string) – the more popular title / the title used by the filmmakers on promotional materials at the point of release
- originalTitle (string) - original title, in the original language
- isAdult (boolean) - 0: non-adult title; 1: adult title
- startYear (YYYY) – represents the release year of a title. In the case of TV Series, it is the series start year
- endYear (YYYY) – TV Series end year. '\\N' for all other title types
- runtimeMinutes – primary runtime of the title, in minutes
- genres (string array) – includes up to three genres associated with the title
</details>

In [3]:
# Setup - Load the SQL extension and connect to the Mini IMDB dataset we've prepared
db_path = Path('data/imdbmini.db')

engine = sqlalchemy.create_engine(f"sqlite:///{db_path}")
connection = engine.connect()
inspector = sqlalchemy.inspect(engine)

#Get table names
inspector.get_table_names()



['Name', 'Rating', 'Role', 'Title']

In [4]:
query_name = """
SELECT * 
FROM Name
"""
pd.read_sql(query_name, engine).head()

,nconst,primaryName,birthYear,deathYear,primaryProfession
0,1,Fred Astaire,1899,1987,"soundtrack,actor,miscellaneous"
1,2,Lauren Bacall,1924,2014,"actress,soundtrack"
2,3,Brigitte Bardot,1934,None,"actress,soundtrack,music_department"
3,4,John Belushi,1949,1982,"actor,soundtrack,writer"
4,5,Ingmar Bergman,1918,2007,"writer,director,actor"


In [5]:
query_role = """
SELECT * 
FROM Rating
"""
pd.read_sql(query_role, engine).head()

,index,tconst,averageRating,numVotes
0,0,417,8.2,46382
1,1,4972,6.3,23836
2,2,10323,8.1,60244
3,3,12349,8.3,120059
4,4,13442,7.9,92218


In [6]:
query_rating = """
SELECT * 
FROM Role
"""
pd.read_sql(query_rating, engine).head()

,index,tconst,ordering,nconst,category,job,characters
0,0,417,1,617588,actor,None,"[""Prof. Barbenfouillis"",""The Moon""]"
1,1,417,2,29244,actor,None,"[""Astronomer""]"
2,2,417,3,76933,actress,None,"[""Lady in the Moon""]"
3,3,417,4,1215996,actor,None,"[""Astronomer""]"
4,4,417,5,894523,writer,"novel ""De la Terre à la Lune""",None


In [7]:
query_title = """
SELECT * 
FROM Title 
LIMIT 3
"""
pd.read_sql(query_title, engine).head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,417,short,A Trip to the Moon,Le voyage dans la lune,0,1902,None,13,"Action,Adventure,Comedy"
1,4972,movie,The Birth of a Nation,The Birth of a Nation,0,1915,None,195,"Drama,History,War"
2,10323,movie,The Cabinet of Dr. Caligari,Das Cabinet des Dr. Caligari,0,1920,None,76,"Fantasy,Horror,Mystery"


## Question 1: Find all *distinct* film roles.

<br/>
<details>
    <summary>▶ [Click to Expand] <b>Hints + Output</b></summary>

- Tests: `DISTINCT`
- Our solution is 2 lines.
- **Hint Data**: The table `Role` contains all film roles.
- **Hint**: Use `DISTINCT`
</details>

The first 3 lines of the table should look like the following (but you should compute the entire table).
    
|category|
|---|
|actor|
|actress |
|writer |

<!--
BEGIN QUESTION
name: q1
points: 1
-->

In [8]:
query_q1 = """
SELECT DISTINCT(category)
FROM Role
"""


res_q1 = pd.read_sql(query_q1, engine)
res_q1.head()

,category
0,actor
1,actress
2,writer
3,composer
4,editor


In [9]:
grader.check("q1")

q1 results: All test cases passed!

## Question 2: Compute number of film releases, annually.

Details: Sort from newest to oldest, and ignore titles with `NULL` years.

<br/>

<details>
    <summary>▶ [Click to Expand] <b>Hints</b></summary>

- Tests: grouping, aggregation, handling NULL
- Our solution is 7 lines.
    
- **Approach**: You need to (#1) filter out `NULL` years, (#2) group by year and (#3) count within each group.
        
- **Hint #1**: For filtering out `NULL` values, see lecture 9.5 @ 2:20 https://youtu.be/LEMkOxv599U?t=140
    
- **Hint #2**: For using `GROUP BY`, see lecture 8.6 @ 2:46 https://youtu.be/WsyA75ppEAU?t=166
    
- **Hint Data**: Per the IMDB Dataset description, use the column `startYear` to determine when a film was released.
    
- **Hint**: For changing `ORDER BY` direction, consider `DESC` (descending) and `ASC` (ascending).
</details>

Output the year `year` and the number of films per year `total`. The first 3 lines of the table should look like the following (but you should compute the first 10 rows).
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th>year</th>
      <th>total</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>2021</td>
      <td>128</td>
    </tr>
    <tr>
      <td>2020</td>
      <td>181</td>
    </tr>
    <tr>
      <td>2019</td>
      <td>278</td>
    </tr>
  </tbody>
</table>

<!--
BEGIN QUESTION
name: q2
points: 2
-->

In [10]:
query_q2 = """
SELECT startYear AS year, count(*) AS total
FROM Title
WHERE year is NOT NULL
GROUP BY year
ORDER BY year DESC
LIMIT 10
"""


res_q2 = pd.read_sql(query_q2, engine)
res_q2.head()

,year,total
0,2021,128
1,2020,181
2,2019,278
3,2018,282
4,2017,284


In [11]:
grader.check("q2")

q2 results: All test cases passed!

## Question 3: Report number of "Big-Hit" and "Not-Big-Hit" films.

Details: A "big hit" is a film with over 100,000 votes.

<br/>

<details>
    <summary>▶ [Click to Expand] <b>Hints</b></summary>

- Tests: Case, one-to-many Join
- Our solution is 10 lines.
    
- **Approach**: You need to (1) link films to ratings, (2) compute whether or not a film is a "hit", according to its ratings, and (3) group by and aggregate within hits and not hits.
    
- **Hint Data**: Use `Rating.numVotes` for number of votes.
- **Hint #1**: Use `JOIN`. See 9.4 @ 3:23 for an example https://youtu.be/FAYKJT1TFUA?t=199
- **Hint #2**: Use `CASE`. See 9.7 @ 0:40 for a brief overview https://youtu.be/6vkZUwkoAwM?t=40
- Review #3: For using `GROUP BY`, see lecture 8.6 @ 2:46 https://youtu.be/WsyA75ppEAU?t=166
   
</details>

Output whether or not the film is a hit `isBigHit` and number of films in that category `total`. Your output should match the following, exactly:

|isBigHit|total|
|---|---|
|no|5341|
|yes|2233|

<!--
BEGIN QUESTION
name: q3
points: 3
-->

In [12]:
query_q3 = """
SELECT
    CASE
        -- you need to convert it to the int datatype
        WHEN CAST(r1.numVotes AS int) > 100000 THEN 'yes'
        ELSE 'no'
    END AS isBigHit,
    count(r2.numVotes) AS total
FROM Rating AS r1
INNER JOIN Rating as r2
WHERE r1.tconst = r2.tconst
GROUP BY isBigHit
"""


res_q3 = pd.read_sql(query_q3, engine)
res_q3

,isBigHit,total
0,no,5341
1,yes,2233


In [13]:
grader.check("q3")

q3 results: All test cases passed!

## Question 4: Compute average rating across films, per year.

**Warning**: `Rating` provides average rating *per film*, but we need average rating *across votes* per year. See hints for details.

<br/>

<details>
    <summary>▶ [Click to Expand] <b>Hints</b></summary>
    
    
- Tests: SQL aggregates + arithmetic
- Our solution is 7 lines
    
- **Approach**: You'll need to (1) link titles with ratings, (2) group by and aggregate within each title's release year, according to the formula above.
    

- **Hint Data**: Use `Title.startYear` for film release year.
- **Hint Data**: You shouldn't use the `AVG(...)` aggregate, because the `Rating` table contains *average ratings per film* and the number of votes. We want the *average across all votes*, for a year of films, which we can compute using: `sumRatingsPerYear / numVotesInYear = sum(averageFilmRating * numFilmVotes) / sum(numFilmVotes)` (the sums are across all films in a single year).
    
- Review #1: Use a `JOIN`. See 9.4 @ 3:23 for an example https://youtu.be/FAYKJT1TFUA?t=199
- Review #2: For using `GROUP BY`, see lecture 8.6 @ 2:46 https://youtu.be/WsyA75ppEAU?t=166. 
</details>

Output the year `year` and average rating `rating`, ordered by `year` with the most recent year at the top. The first 3 lines of your output should look like the following (but you should compute the first 10 rows):

|year|rating|
|---|---|
|2021|7.036896|
|2020|7.004193|
|2019|7.333072|

<!--
BEGIN QUESTION
name: q4
points: 3
-->

In [14]:
query_q4 = """
SELECT 
    t.startYear AS year, 
    SUM(r.averageRating * r.numVotes) / SUM(r.numVotes) AS rating
FROM Title AS t, Rating as r
WHERE t.tconst = r.tconst
GROUP BY year
ORDER BY year DESC
LIMIT 10
"""


res_q4 = pd.read_sql(query_q4, engine)
res_q4.head()

,year,rating
0,2021,7.036896
1,2020,7.004193
2,2019,7.333072
3,2018,7.083549
4,2017,7.196795


In [15]:
grader.check("q4")

q4 results: All test cases passed!

## Question 5: Find people with the most ratings.

Details: Sort from highest to lowest.

<br/>

<details>
    <summary>▶ [Click to Expand] <b>Hints</b></summary>
    
- Tests: many-to-many join
- Our solution is 13 lines, excluding the `VIEW` scaffolding.
- **Approach**: You'll need to (1) link name to ratings, (2) group all ratings per person, and (3) compute total number of votes.
- **Hint #1**: Use multiple `JOIN`s. Here is an example with multiple JOINs (but you may want to watch the entire lesson from the start) 9.7 @ 5:32 https://youtu.be/6vkZUwkoAwM?t=332
- Review #2: For using GROUP BY, see lecture 8.6 @ 2:46 https://youtu.be/WsyA75ppEAU?t=166.
</details>

Output the person's `name`, average rating `rating`, and number of votes cast `numVotes`. The first 3 lines of your output should look like the following:

|nconst|name|numVotes|
|---|---|---|
|2369|John Williams|16574610|
|93|Brad Pitt|16007625|
|138|Leonardo DiCaprio|14757121|

<!--
BEGIN QUESTION
name: q5
points: 2
-->

In [16]:
query_q5 = """
CREATE VIEW PeopleRating AS
SELECT
    n.nconst,
    n.primaryName AS name,
    SUM(r.numVotes) AS numVotes
FROM Name AS n 
INNER JOIN Role AS c
ON n.nconst = c.nconst             -- (nconst, name) pair
JOIN Rating AS r
ON r.tconst = c.tconst
GROUP BY name
ORDER BY numVotes DESC
"""


#writing PeopleRating to db
engine.execute("DROP VIEW IF EXISTS PeopleRating")
engine.execute(query_q5)

res_q5 = pd.read_sql("SELECT * FROM PeopleRating LIMIT 10", engine)
res_q5

,nconst,name,numVotes
0,2369,John Williams,16574610
1,93,Brad Pitt,16007625
2,138,Leonardo DiCaprio,14757121
3,158,Tom Hanks,14703255
4,498278,Stan Lee,13906803
5,634240,Christopher Nolan,13728818
6,229,Steven Spielberg,13610887
7,134,Robert De Niro,13415268
8,168,Samuel L. Jackson,11997152
9,288,Christian Bale,11892101


In [17]:
grader.check("q5")

q5 results: All test cases passed!


## Question 6: Compute number of A/B/C-list people per film.

Details: A-list people have 1,000,000+ votes. B-list have 500,000 - 999,999 votes. C-list have 100,000 - 499,999. Return top 10 films, ranked by number of A-list actors/actresses.

Before looking below, take a moment to consider how you would do this problem. Would you use a `CASE`? (Why can't you?)

### Question 6a: Compute "X-List" classification per person.

<br/>

<details>
    <summary>▶ [Click to Expand] <b>Hints</b></summary>
    
- Tests: `AND` for predicates
- The solution is 7 lines.
</details>

Using `PeopleRating` from Question 5, compute booleans `isAList`, `isBList`, and `isCList` for each person. Additionally output `PeopleRating.nconst`. The first 3 lines of your output should look like the following (but when using this query for Question 6, compute the full table).

|isAList|isBList|isCList|nconst|
|---|---|---|---|
|1|0|0|2369|
|1|0|0|93|
|1|0|0|138|

<!--
BEGIN QUESTION
name: q6a
points: 2
-->

In [18]:
query_q6a = """
CREATE VIEW PeopleABC AS
SELECT
    CASE
        WHEN numVotes > 1000000 THEN 1
        ELSE 0
    END AS isAList,
    CASE
        WHEN numVotes >= 500000 AND numVotes <= 999999 THEN 1
        ELSE 0
    END AS isBList,
    CASE 
        WHEN numVotes >= 100000 AND numVOtes <= 499999 THEN 1
        ELSE 0
    END AS isCList,
    nconst
FROM PeopleRating 
ORDER BY isAList DESC
"""


#writing PeopleABC to db
engine.execute("DROP VIEW IF EXISTS PeopleABC")
engine.execute(query_q6a)

res_q6a = pd.read_sql("SELECT * FROM PeopleABC LIMIT 10", engine)
res_q6a

,isAList,isBList,isCList,nconst
0,1,0,0,2369
1,1,0,0,93
2,1,0,0,138
3,1,0,0,158
4,1,0,0,498278
5,1,0,0,634240
6,1,0,0,229
7,1,0,0,134
8,1,0,0,168
9,1,0,0,288


In [19]:
grader.check("q6a")

q6a results: All test cases passed!

### Question 6b: Count up "X-List" people per film

<br/>

<details>
    <summary>▶ [Click to Expand] <b>Hints</b></summary>
    
- Tests: When NOT to use case
    
- **Approach**: You need to: (1) Use a subquery, common table expression, or view to compute a boolean column for each "rank" -- `isAList`, `isBList` and `isCList` for every person; (2) link people ranks with titles; (3) group by and sum within each movie.
    
- **Hint #1**: Try Question 6a if this is confusing.
- Reivew #2: Use multiple JOINs. Here is an example with multiple JOINs (but you may want to watch the entire lesson from the start) 9.7 @ 5:32 https://youtu.be/6vkZUwkoAwM?t=332
- Review #3: For using GROUP BY, see lecture 8.6 @ 2:46 https://youtu.be/WsyA75ppEAU?t=166.
</details>

Output the number of A-list `numAList`, B-list `numBList`, and C-list `numCList` people, along with the film title. The first 3 lines of your output should look like the following (but you should compute the first 10 rows):

|numAList|	numBList|	numCList|	primaryTitle|
|---|---|---|---|
|10|	0|	0|	The Godfather |
|10|	0|	0|	The Godfather: Part II |
|10|	0|	0|	Star Wars: Episode V - The Empire Strikes Back |

<!--
BEGIN QUESTION
name: q6b
points: 3
-->

In [20]:
query_q6b = """
SELECT 
    SUM(p.isAList) AS numAList,
    SUM(p.isBList) AS numBList,
    SUM(p.isCList) AS numCList,
    primaryTitle
FROM PeopleABC as p
INNER JOIN Role as r
ON p.nconst = r.nconst
INNER JOIN Title as t
ON r.tconst = t.tconst
GROUP BY t.tconst
ORDER BY numAList DESC
LIMIT 10
"""


res_q6b = pd.read_sql(query_q6b, engine)
res_q6b

,numAList,numBList,numCList,primaryTitle
0,10,0,0,The Godfather
1,10,0,0,The Godfather: Part II
2,10,0,0,Star Wars: Episode V - The Empire Strikes Back
3,10,0,0,Back to the Future
4,10,0,0,Die Hard
5,10,0,0,Goodfellas
6,10,0,0,The Silence of the Lambs
7,10,0,0,Terminator 2: Judgment Day
8,10,0,0,A Few Good Men
9,10,0,0,Forrest Gump


In [21]:
grader.check("q6b")

q6b results: All test cases passed!

## Question 7: Find pairs of people that have 50+ collaborations.

Before looking below, take a moment to consider how you would do this problem. Can you do this without subqueries or common table expressions? Why not?

### Question 7a: For each movie, list all pairs of collaborators.

**Warning**: Make sure not to double-count pairs or to match any person with themselves as a "pair". See hints for details.

<br/>

<details>
    <summary>▶ [Click to Expand] <b>Hints</b></summary>

- Testing: self-joins with pairs

- Our solution has 8 lines excluding the `VIEW` scaffolding.
    
- **Approach**: You will need to (1) consider pairs and (2) filter pairs to avoid double-counting and/or counting one person with themselves as a pair.
    
- **Hint #1**: Pick whichever approach is more intuitive. You can (a) compute all pairs, then filter out invalid pairs using hint #2 like 9.4 @ 4:20 https://youtu.be/FAYKJT1TFUA?t=254 OR (b) write the more efficient inner JOIN, using hint #2 as the join predicate, like 9.4 @ 3:19 https://youtu.be/FAYKJT1TFUA?t=199
- **Hint #2**: To avoid double-counting and avoid counting one person with themselves as a pair, see 9.7 @ 7:19 https://youtu.be/6vkZUwkoAwM?t=439 
</details>

Output primary keys for both people `nconst1`, `nconst2` and for the film `tconst`. The first 3 lines of your output should look like the following (but you should compute the first 10 rows)

|nconst1	|nconst2	|tconst|
|---|---|---|
|617588	|29244	|417|
|617588	|76933	|417|
|617588	|242155	|417|

<!--
BEGIN QUESTION
name: q7a
points: 2
-->

In [22]:
query_q7a = """
CREATE VIEW Pair AS
SELECT
    r1.nconst AS nconst1,
    r2.nconst AS nconst2,
    r1.tconst
FROM Role AS r1 INNER JOIN Role AS r2
ON r1.tconst = r2.tconst
WHERE nconst1 > nconst2
"""


#writing Pair to db
engine.execute("DROP VIEW IF EXISTS Pair")
engine.execute(query_q7a)

res_q7a = pd.read_sql("SELECT * FROM Pair LIMIT 10", engine)
res_q7a.head()

,nconst1,nconst2,tconst
0,617588,29244,417
1,617588,76933,417
2,617588,242155,417
3,617588,324073,417
4,76933,29244,417


In [23]:
grader.check("q7a")

q7a results: All test cases passed!

### Question 7b: Report number of movies per pair.

Details: For all pairs that have collaborated at least 50 times.

<br/>

<details>
    <summary>▶ [Click to Expand] <b>Hints</b></summary>

- Tests: group by over multiple columns
- Our solution has 12 lines.
</details>

Output the names of both people, `name1` and `name2`, in addition to the number of collaborations `total`. The first 3 lines of your output should look like the following (but you should compute the whole table)

|name1|	name2|	total|
|---|---|---|
|D.B. Weiss	|David Benioff|	74|
|David Benioff|	George R.R. Martin|	73|
|D.B. Weiss|	George R.R. Martin|	73|

<!--
BEGIN QUESTION
name: q7b
points: 3
-->

In [24]:
query_q7b = """
SELECT 
    n1.primaryName AS name1,
    n2.primaryName AS name2,
    COUNT(*) AS total
FROM Pair AS p
INNER JOIN Name AS n1
ON n1.nconst = p.nconst1
INNER JOIN Name AS n2
ON n2.nconst = p.nconst2
GROUP BY p.nconst1, p.nconst2
HAVING total >= 50
ORDER BY total DESC
"""


res_q7b = pd.read_sql(query_q7b, engine)
res_q7b.head()

,name1,name2,total
0,D.B. Weiss,David Benioff,74
1,David Benioff,George R.R. Martin,73
2,D.B. Weiss,George R.R. Martin,73
3,David Benioff,Peter Dinklage,59
4,D.B. Weiss,Peter Dinklage,59


In [25]:
grader.check("q7b")

q7b results: All test cases passed!

## [Optional] Question 8: Find person with the highest ratings improvement, from first to last year of films.

**Before looking at the code below**, I suggest thinking about how you would approach this question.

<br/>

<details>
    <summary>▶ [Click to Expand] <b>See how I approached this.</b></summary>
    
I broke up this problem into 2 subqueries/views/CTEs, in addition to the actual query:
1. `PeopleAnnualRating`: Compute the annual rating per person. Link all the relevant tables, reuse the average rating formula from before, and do a joint group by over both names and film years.
2. `FirstLastYear`: Compute the first and last year that each person was involved in the film industry.
3. Join `FirstLastYear` with the relevant rows from `PeopleAnnualRating` to get ratings for each person's first and last years
</details>

**This is written as a debug question.** Your goal is to find what is wrong and fix it. What happens is: If you uncomment the last cell in this notebook and run it, the cell will run forever and crash the kernel. Why is that? The answer is in the next collapsible. See if you can come up with the fix on your own first.

<br/>

<details>
    <summary>▶ [Click to Expand] <b>See the answer for how to fix.</b></summary>

1. The query running forever is a sign that we're producing a combinatorially large table. So that immediately narrowed down my focus to the join predicates `ON...`.
2. Looking there, I realized that I had forgotten to filter by the name. Forgetting that meant I was joining `FirstLastYear` with *all* people that had ratings in `PeopleAnnualRating` for a single year.... which is to say, a lot of people! And I was doing this for each person in `FirstLastYear`.
3. So, the answer is just to additionally add a name filter to the predicate for both JOINs, like this `LastYear.year = FirstLastYear.lastYear AND LastYear.nconst = FirstLastYear.nconst`
</details>

In [26]:
query_q8a = """
CREATE VIEW PeopleAnnualRating AS
    SELECT 
        Role.nconst,
        Title.startYear AS year,
        SUM(Rating.averageRating * Rating.numVotes) / SUM(Rating.numVotes) AS rating
    FROM Role
    JOIN Title
        ON Role.tconst = Title.tconst
    JOIN Rating
        ON Rating.tconst = Title.tconst
    GROUP BY Role.nconst, Title.startYear
"""

#writing PeopleAnnualRating to db
engine.execute("DROP VIEW IF EXISTS PeopleAnnualRating")
engine.execute(query_q8a)

res_q8a = pd.read_sql("SELECT * FROM PeopleAnnualRating LIMIT 3", engine)
res_q8a.head()

,nconst,year,rating
0,1,1957,7.0
1,2,1944,7.8
2,2,1946,7.9


In [27]:
query_q8b = """
CREATE VIEW FirstLastYear AS
    SELECT
        MIN(year) AS firstYear,
        MAX(year) AS lastYear,
        nconst
    FROM PeopleAnnualRating
    GROUP BY nconst
"""

#writing PeopleAnnualRating to db
engine.execute("DROP VIEW IF EXISTS FirstLastYear")
engine.execute(query_q8b)

res_q8b = pd.read_sql("SELECT * FROM FirstLastYear LIMIT 3", engine)
res_q8b.head()

,firstYear,lastYear,nconst
0,1957,1957,1
1,1944,2004,2
2,1963,1963,3


In [28]:
# fix this query
# query_q8c = """
# SELECT
#     LastYear.rating - StartYear.rating AS improvement,
#     Name.primaryName
# FROM FirstLastYear
# JOIN PeopleAnnualRating AS StartYear
#     ON StartYear.year = FirstLastYear.firstYear
# JOIN PeopleAnnualRating AS LastYear
#     ON LastYear.year = FirstLastYear.lastYear
# JOIN Name
#     ON Name.nconst = FirstLastYear.nconst
# ORDER BY improvement DESC
# LIMIT 10;
# """

query_q8c = """
SELECT
    LastYear.rating - StartYear.rating AS improvement,
    Name.primaryName
FROM FirstLastYear
JOIN PeopleAnnualRating AS StartYear
    ON StartYear.year = FirstLastYear.firstYear AND StartYear.nconst = FirstLastYear.nconst
JOIN PeopleAnnualRating AS LastYear
    ON LastYear.year = FirstLastYear.lastYear AND LastYear.nconst = FirstLastYear.nconst
JOIN Name
    ON Name.nconst = FirstLastYear.nconst
ORDER BY improvement DESC
LIMIT 10;
"""


res_q8c = pd.read_sql(query_q8c, engine)
res_q8c.head()

,improvement,primaryName
0,5.8,Bobby Deol
1,5.5,Ranveer Singh
2,5.2,Ming-Na Wen
3,5.0,Mariah Carey
4,4.5,Billy Brown


---

To double-check your work, the cell below will rerun all of the autograder tests.

In [29]:
grader.check_all()

q1 results: All test cases passed!

q2 results: All test cases passed!

q3 results: All test cases passed!

q4 results: All test cases passed!

q5 results: All test cases passed!

q6a results: All test cases passed!

q6b results: All test cases passed!

q7a results: All test cases passed!

q7b results: All test cases passed!

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

In [30]:
# Save your notebook first, then run this cell to export your submission.
grader.export()

There was an error generating your LaTeX; showing full error message:
This is BibTeX, Version 0.99d (TeX Live 2021)
The top-level auxiliary file: notebook.aux
I found no \citation commands---while reading file notebook.aux
I found no \bibdata command---while reading file notebook.aux
I found no \bibstyle command---while reading file notebook.aux
(There were 3 error messages)

If the error above is related to xeCJK or fandol in LaTeX and you don't require this functionality, try running again with no_xecjk set to True or the --no-xecjk flag.


/usr/local/Caskroom/miniconda/base/envs/mixed/lib/python3.9/site-packages/otter/check/notebook.py:328: UserWarning: Could not locate a PDF to include
  warnings.warn("Could not locate a PDF to include")
